# Import Libraries and Construct Data

In [37]:
from __future__ import division
import pickle
import random
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

In [38]:
auc_results = []
recall_results = []

In [39]:
seed = [76424236, 35427387, 90786253, 84636361, 61510520]
split = 4

np.random.seed(seed=seed[split])

folder_name = f"./Pre-Process-Split/Split_data/split_{split}/"

In [40]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['test_data_pos'] = np.load(folder_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load(folder_name + "biased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

# Define & Load Model

In [41]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [42]:
cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

In [43]:
cml_model.load(folder_name+"cml-yahoo")

INFO:tensorflow:Restoring parameters from ./Split_data/split_4/cml-yahoo


# Generate Raw Results

In [44]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 15400/15400 [05:39<00:00, 45.31it/s]


{'AUC': [0.4365646258503401,
  0.5128401360544218,
  0.5184693877551021,
  0.4634333333333334,
  0.47164429530201346,
  0.4895986622073579,
  0.4955460750853242,
  0.473236301369863,
  0.5107357859531771,
  0.5133277591973244,
  0.4274333333333334,
  0.48194256756756765,
  0.4658922558922559,
  0.47572390572390577,
  0.46171768707482996,
  0.49887205387205386,
  0.50515,
  0.48677474402730375,
  0.47255892255892257,
  0.486547619047619,
  0.4918624161073825,
  0.4991333333333334,
  0.45951342281879193,
  0.4657000000000001,
  0.4831678082191781,
  0.5417966101694915,
  0.4877946127946128,
  0.4511244019138756,
  0.5371070234113713,
  0.5122586206896552,
  0.49905,
  0.5092307692307692,
  0.4955333333333333,
  0.49148648648648646,
  0.496026936026936,
  0.5050841750841751,
  0.43799331103678935,
  0.47191666666666665,
  0.51815,
  0.5360367892976587,
  0.5141166666666667,
  0.5328691275167786,
  0.5120777027027027,
  0.5522742474916389,
  0.48405000000000004,
  0.5374402730375427,
  0.4

# Evaluation

In [45]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=30):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 30, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [46]:
#eq("bpr-yahoo-test-pos-biased_evaluate_partial.pickle", "bpr-yahoo-test-neg-biased_evaluate_partial.pickle", "training_arr.npy", gamma=1.5)

In [52]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5)

{'auc': 0.8457143376264993, 'recall': 0.5961488493994731}

In [50]:
#Key number of split

results_cml = {
    0: {'auc': 0.8460377901121725, 'recall': 0.5966479961429412},
    1 : {'auc': 0.8444201772869976, 'recall': 0.5961154773708655},
    2 : {'auc': 0.8449645966277645, 'recall': 0.5942566819729325},
    3 : {'auc': 0.8458186513100918, 'recall': 0.5970229872081129},
    4 : {'auc': 0.8457143376264993, 'recall': 0.5961488493994731}
}

In [51]:
auc_values = [results_cml[i]['auc'] for i in results_cml]
recall_values = [results_cml[i]['recall'] for i in results_cml]

average_auc = sum(auc_values) / len(auc_values)
average_recall = sum(recall_values) / len(recall_values)

average_auc, average_recall


(0.8453911105927052, 0.5960383984188651)